In [26]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols, Dummy, lambdify
from sympy.physics import mechanics
from scipy.integrate import odeint
from matplotlib import animation

In [27]:
def integrate_inverted_pendulum(n, times, initial_positions=135, initial_velocities=0, lengths=None, masses=1):
    """Integrate an inverted multi-pendulum with `n` sections"""
    
    # Step 1: construct the pendulum model
    q = mechanics.dynamicsymbols('q:{0}'.format(n))
    u = mechanics.dynamicsymbols('u:{0}'.format(n))
    m = symbols('m:{0}'.format(n))
    l = symbols('l:{0}'.format(n))
    g, t = symbols('g,t')
    
    # Step 2: build the model using Kane's Method
    A = mechanics.ReferenceFrame('A')
    P = mechanics.Point('P')
    P.set_vel(A, 0)
    
    particles = []
    forces = []
    kinetic_odes = []
    
    for i in range(n):
        Ai = A.orientnew('A' + str(i), 'Axis', [q[i], A.z])
        Ai.set_ang_vel(A, u[i] * A.z)
        Pi = P.locatenew('P' + str(i), l[i] * Ai.x)
        Pi.v2pt_theory(P, A, Ai)
        Pai = mechanics.Particle('Pa' + str(i), Pi, m[i])
        particles.append(Pai)
        forces.append((Pi, -m[i] * g * A.x))  # Negative sign for inverted pendulum
        kinetic_odes.append(q[i].diff(t) - u[i])
        P = Pi

    KM = mechanics.KanesMethod(A, q_ind=q, u_ind=u, kd_eqs=kinetic_odes)
    fr, fr_star = KM.kanes_equations(particles, forces)
    
    # Step 3: numerically evaluate equations and integrate
    y0 = np.deg2rad(np.concatenate([np.broadcast_to(initial_positions, n),
                                    np.broadcast_to(initial_velocities, n)]))
        
    if lengths is None:
        lengths = np.ones(n) / n
    lengths = np.broadcast_to(lengths, n)
    masses = np.broadcast_to(masses, n)
    
    parameters = [g] + list(l) + list(m)
    parameter_vals = [9.81] + list(lengths) + list(masses)
    unknowns = [Dummy() for i in q + u]
    unknown_dict = dict(zip(q + u, unknowns))
    kds = KM.kindiffdict()
    
    mm_sym = KM.mass_matrix_full.subs(kds).subs(unknown_dict)
    fo_sym = KM.forcing_full.subs(kds).subs(unknown_dict)
    
    mm_func = lambdify(unknowns + parameters, mm_sym)
    fo_func = lambdify(unknowns + parameters, fo_sym)

    def gradient(y, t, args):
        vals = np.concatenate((y, args))
        sol = np.linalg.solve(mm_func(*vals), fo_func(*vals))
        return np.array(sol).T[0]

    return odeint(gradient, y0, times, args=(parameter_vals,))

In [28]:
def get_xy_coords(p, lengths=None):
    p = np.atleast_2d(p)
    n = p.shape[1] // 2
    if lengths is None:
        lengths = np.ones(n) / n
    zeros = np.zeros(p.shape[0])[:, None]
    x = np.hstack([zeros, lengths * np.sin(p[:, :n])])
    y = np.hstack([zeros, -lengths * np.cos(p[:, :n])])
    return np.cumsum(x, 1), np.cumsum(y, 1)

In [29]:
def animate_inverted_pendulum(n):
    t = np.linspace(0, 10, 200)
    p = integrate_inverted_pendulum(n, t, initial_positions=180)
    x, y = get_xy_coords(p)
    
    fig, ax = plt.subplots(figsize=(6, 6))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    ax.axis('off')
    ax.set(xlim=(-1, 1), ylim=(-1, 1))
    line, = ax.plot([], [], 'o-', lw=2)

    def init():
        line.set_data([], [])
        return line,

    def animate(i):
        line.set_data(x[i], y[i])
        return line,

    anim = animation.FuncAnimation(fig, animate, frames=len(t),
                                   interval=1000 * t.max() / len(t),
                                   blit=True, init_func=init)
    plt.close(fig)
    return anim

In [30]:
anim = animate_inverted_pendulum(2)
from IPython.display import HTML
HTML(anim.to_html5_video())